In [2]:
from NN_Thesis.nn_classes import *
from NN_Thesis.trainer import *
from NN_Thesis.adapters import *
from NN_Thesis.helper_functions import total_elem
from NN_Thesis.models.ReactNet import *
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
from torch import optim


from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import numpy as np
import os
from PIL import Image
import random

from matplotlib import pyplot as plt

import wandb
import torch.nn.utils.prune as prune



In [9]:
path = r'NN_Thesis\pretrained\model_best_resnet.pth.tar'
state = torch.load(path)

new_dict = {}


net_dict =state['state_dict']
for key in net_dict.keys():
    split_key = key.split('.')
    new_key = '.'.join(split_key[1:])
    # print(new_key)
    # weight = getattr(getattr(net,split_key[1]),split_key[-1])
    # print(weight.shape)
    # assert net_dict[key].shape == weight.shape
    new_dict[new_key] = net_dict[key]

print(state.keys())

torch.save(new_dict,r'NN_Thesis\pretrained\Pretrained_BiReal_Model.pth')

dict_keys(['epoch', 'state_dict', 'best_top1_acc', 'optimizer'])


In [6]:
def total_elem(net,per_layer = False):
    total = 0
    bin_total = 0
    for name,m in net.named_modules():
        for p in m.parameters():
            if isinstance(c,BinarizeConv2d) or isinstance(c,BinarizeLinear):
                bin_total += p.numel()
            total += p.numel()

    print(f'Total Number of Elements: {total}, Binary Elements {bin_total}, Ratio = {bin_total/total}')
    return total


net = ResNet_cifar10()

total_elem(net)

BinarizeConv2d(3, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Hardtanh(min_val=-1.0, max_val=1.0, inplace=True)
Hardtanh(min_val=-1.0, max_val=1.0, inplace=True)
Sequential(
  (0): BasicBlock(
    (conv1): BinarizeConv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (tanh1): Hardtanh(min_val=-1.0, max_val=1.0, inplace=True)
    (conv2): BinarizeConv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (tanh2): Hardtanh(min_val=-1.0, max_val=1.0, inplace=True)
    (bn2): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): BinarizeConv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_runni

4338030

# Flower 102

In [3]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop((224,224)),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean,std)
])
valid_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize(mean,std)
])

Flower_Train = torchvision.datasets.Flowers102(root = './data/Flowers102/',split='train',download=True,transform= train_transform)
Flower_Valid = torchvision.datasets.Flowers102(root = './data/Flowers102/',split='val',download=True,transform= valid_transform)

Flower_TrainDL = DataLoader(Flower_Train,batch_size=64,shuffle=True,num_workers=8)
Flower_ValidDL = DataLoader(Flower_Valid,batch_size=64,num_workers=8)

## Regular 

In [4]:
net = birealnet18_uniadapt()
net.load_state_dict(new_dict,strict= False)

_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])

In [5]:
trainer = Trainer(net,model_name='BiReact_Finetune',project_name='Flower102',binarise=False,classes= list(set(Flower_Train._labels)))
trainer.lr = 1e-3
trainer.batch_size = 64
trainer.epochs =100
trainer.epoch_chkpts = []
trainer.start_epoch = 0
# params = nn.ModuleList([BNN.head_bn,BNN.head]).parameters()
trainer.set_scheduler(None)
trainer.set_optimizer(torch.optim.Adam,lr = trainer.lr)
trainer.train(Flower_TrainDL,Flower_ValidDL)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]}]}


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: johnny_suu. Use `wandb login --relogin` to force relogin


start
Project Name: Flower102, Run Name BiReact_Finetune 


Run Start : 2023-04-24 23-52-51
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 100
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 11808974
Initial accuracy:
Test Accuracy : 0.2%, Test Loss: 13.79730623960495
best_acc.pth saved!
Test Accuracy : 0.0%, Test Loss: 14.986623883247375
epoch: 1 average loss: 8.436
Test Accuracy : 1.5%, Test Loss: 7.441400408744812
best_acc.pth saved!
Epoch Time (Training + Test) = 16.72 seconds
epoch: 2 average loss: 6.340
Test Accuracy : 3.2%, Test Loss: 6.100608617067337
best_acc.pth saved!
Epoch Time (

In [10]:
net = birealnet18_uniadapt()
net.load_state_dict(new_dict,strict= False)
net.freeze()
net.fc = nn.Linear(512,102)
net.fc.requires_grad_(True)
trainer = Trainer(net,model_name='BiReact_Finetune_HeadOnly',project_name='Flower102',binarise=False,classes= list(set(Flower_Train._labels)))
trainer.lr = 1e-4
trainer.batch_size = 64
trainer.epochs =75
trainer.epoch_chkpts = []
trainer.start_epoch = 0
# params = nn.ModuleList([BNN.head_bn,BNN.head]).parameters()
trainer.set_scheduler(None)
params = net.fc.parameters()
trainer.set_optimizer(torch.optim.Adam,params,lr = trainer.lr)
trainer.train(Flower_TrainDL,Flower_ValidDL)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.0001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1]}]}


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: johnny_suu. Use `wandb login --relogin` to force relogin


start
Project Name: Flower102, Run Name BiReact_Finetune_HeadOnly 


Run Start : 2023-04-25 10-47-26
start_epoch : 0
initial_lr : 0.0001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.0001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 11348300
Initial accuracy:
Test Accuracy : 1.0%, Test Loss: 4.646392643451691
best_acc.pth saved!
Test Accuracy : 0.5%, Test Loss: 4.648427069187164
epoch: 1 average loss: 4.633
Test Accuracy : 0.8%, Test Loss: 4.622403681278229
Epoch Time (Training + Test) = 15.46 seconds
epoch: 2 average loss: 4.613
Test Accuracy : 1.8%, Test Loss: 4.597882151603699
best_acc.pth saved!
Epoch Time (Training + 

## UniAdapt FineTune

In [4]:

for prune_amount in [0,0.7,0.8,0.9]:

    net = birealnet18_uniadapt()
    net.load_state_dict(new_dict,strict= False)

    adaptnet = uniAdapt_Net_React([64,128,256,512],[64,128,256,512],block = thinBlock2)
    x = torch.rand((3,3,224,224))
    net.freeze()
    net.uniAdapt = True
    net.uniAdaptNet = adaptnet
    net.head.requires_grad_(True)
    net.head.train()

    trainer = Trainer(net,model_name=f'BiReact_UniAdapt_thinblock2_prune amount {prune_amount*100:.0f}%',project_name='Flower102',binarise=False,classes= list(set(Flower_Train._labels)))
    trainer.lr = 1e-4
    trainer.batch_size = 64
    trainer.epochs =75
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.amount = [prune_amount,0]
    trainer.model_to_prune = net.uniAdaptNet
    params = nn.ModuleList([net.head,net.uniAdaptNet]).parameters()
    trainer.set_scheduler(None)
    trainer.set_optimizer(torch.optim.Adam,params,lr = trainer.lr)
    prune_strat = None if prune_amount == 0 else 'oneshot'
    trainer.train(Flower_TrainDL,Flower_ValidDL,prune_strat=prune_strat)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.0001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]}]}


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: johnny_suu. Use `wandb login --relogin` to force relogin


start
Project Name: Flower102, Run Name BiReact_UniAdapt_thinblock2_prune amount 0% 


Run Start : 2023-04-25 08-50-54
start_epoch : 0
initial_lr : 0.0001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.0001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 11823758
Initial accuracy:
Test Accuracy : 0.8%, Test Loss: 4.642065644264221
best_acc.pth saved!
Test Accuracy : 1.0%, Test Loss: 4.6452600955963135
best_acc.pth saved!
epoch: 1 average loss: 4.625
Test Accuracy : 1.8%, Test Loss: 4.604857563972473
best_acc.pth saved!
Epoch Time (Training + Test) = 15.30 seconds
epoch: 2 average loss: 4.564
Test Accuracy : 3.7%, Test Loss: 4.

Current Best Acc,▁▁▁▂▃▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),█▄▅▅▆▃▃▃▄▃▃▅▇▃▃▃▅▆▅▄▆█▄▄▃▂▃▄▄▅▅▅▄▂▂▁▄▆▅▇
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▁▁▂▃▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████
test_loss,███▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
training_loss,███▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
Best Accuracy,0.71667
Current Best Acc,0.71667
Total Time (hours),0.63678
epoch,75


start
Project Name: Flower102, Run Name BiReact_UniAdapt_thinblock2_prune amount 70% 


Run Start : 2023-04-25 09-10-21
start_epoch : 0
initial_lr : 0.0001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.0001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 11823758
Initial accuracy:
Test Accuracy : 0.7%, Test Loss: 4.642391473054886
best_acc.pth saved!
Test Accuracy : 1.4%, Test Loss: 4.639023423194885
best_acc.pth saved!
epoch: 1 average loss: 4.614
Test Accuracy : 1.9%, Test Loss: 4.5941992700099945
best_acc.pth saved!
Epoch Time (Training + Test) = 16.05 seconds
epoch: 2 average loss: 4.545
Test Accuracy : 4.4%, Test Loss: 4

Current Best Acc,▁▁▁▂▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▇█▆█▆▅▆▄▅▄▅▂▁▂▂▁▂▁▂▂▂▂▃▂▂▂▂▁▁▂▂▁▁▂▁▁▁▂▁▂
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▁▂▂▃▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█▇█████████████
test_loss,███▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
training_loss,███▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
Best Accuracy,0.70098
Current Best Acc,0.70098
Total Time (hours),0.62376
epoch,75


start
Project Name: Flower102, Run Name BiReact_UniAdapt_thinblock2_prune amount 80% 


Run Start : 2023-04-25 09-29-32
start_epoch : 0
initial_lr : 0.0001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.0001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 11823758
Initial accuracy:
Test Accuracy : 0.2%, Test Loss: 4.63825911283493
best_acc.pth saved!
Test Accuracy : 0.9%, Test Loss: 4.64080998301506
best_acc.pth saved!
epoch: 1 average loss: 4.628
Test Accuracy : 2.8%, Test Loss: 4.597910791635513
best_acc.pth saved!
Epoch Time (Training + Test) = 14.80 seconds
epoch: 2 average loss: 4.563
Test Accuracy : 5.5%, Test Loss: 4.54

Current Best Acc,▁▁▂▃▄▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▂▆▆▄▅▃▃▄▅▄▄▄▅▃▃▂▂▄▂█▁▃▂▃▁▃▄▃▄▂▄▃▄▇▂▃▄▂▁▅
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▁▂▃▄▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇██████████
test_loss,███▇▇▇▆▆▆▆▅▅▅▄▄▄▄▄▃▃▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
training_loss,███▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
Best Accuracy,0.68922
Current Best Acc,0.68922
Total Time (hours),0.60816
epoch,75


start
Project Name: Flower102, Run Name BiReact_UniAdapt_thinblock2_prune amount 90% 


Run Start : 2023-04-25 09-48-12
start_epoch : 0
initial_lr : 0.0001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.0001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 11823758
Initial accuracy:
Test Accuracy : 0.5%, Test Loss: 4.633132129907608
best_acc.pth saved!
Test Accuracy : 0.2%, Test Loss: 4.63743382692337
epoch: 1 average loss: 4.622
Test Accuracy : 1.6%, Test Loss: 4.594659894704819
best_acc.pth saved!
Epoch Time (Training + Test) = 14.57 seconds
epoch: 2 average loss: 4.556
Test Accuracy : 3.7%, Test Loss: 4.540223121643066
best_

In [7]:
for prune_amount in [0,0.7,0.8,0.9,0.95]:

    net = birealnet18_uniadapt()
    net.load_state_dict(new_dict,strict= False)

    adaptnet = uniAdapt_Net_React([64,128,256,512],[64,128,256,512],block = thinBlock3)
    x = torch.rand((3,3,224,224))
    net.freeze()
    net.uniAdapt = True
    net.uniAdaptNet = adaptnet
    net.head.requires_grad = True
    net.head.train()

    trainer = Trainer(net,model_name=f'BiReact_UniAdapt_thinblock3_prune amount {prune_amount*100:.0f}%',project_name='Flower102',binarise=False,classes= list(set(Flower_Train._labels)))
    trainer.lr = 1e-3
    trainer.batch_size = 64
    trainer.epochs =75
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.amount = [0.7,prune_amount]
    trainer.model_to_prune = net.uniAdaptNet
    params = nn.ModuleList([net.head,net.uniAdaptNet]).parameters()
    trainer.set_scheduler(None)
    trainer.set_optimizer(torch.optim.Adam,params,lr = trainer.lr)
    prune_strat = None if prune_amount == 0 else 'oneshot'
    trainer.train(Flower_TrainDL,Flower_ValidDL,prune_strat=prune_strat)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]}]}


Current Best Acc,▁▁▁▂▃▃▄▅▅▆▆▇▇███████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▄▃▄▃▄▁▆▃▅▃▂▃▃▂▇▄▂▁▃▄▄▁█▄▃▂▂▄▁▂▄▄▂▂▅▆▄▄▁▅
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▁▂▂▃▄▅▅▅▆▆▇▇▇█▇▇███▂▄▅▅▅▅▅▆▆▆▆▆▇▆▇▇▇▇▇▇
test_loss,██▇▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▆▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂
training_loss,██▇▇▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▅▅▅▅▄▄▄▄▄▃▃▃▂▃▂▂▂▂▂▂
Best Accuracy,0.10784
Current Best Acc,0.10784
Total Time (hours),0.81945
epoch,100


start
Project Name: Flower102, Run Name BiReact_UniAdapt_thinblock3_prune amount 0% 


Run Start : 2023-04-25 02-00-28
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12256526
Initial accuracy:
Test Accuracy : 0.7%, Test Loss: 4.634344309568405
best_acc.pth saved!
Test Accuracy : 0.4%, Test Loss: 4.63129535317421
epoch: 1 average loss: 4.322
Test Accuracy : 23.5%, Test Loss: 3.94725638628006
best_acc.pth saved!
Epoch Time (Training + Test) = 14.64 seconds
epoch: 2 average loss: 3.665
Test Accuracy : 40.6%, Test Loss: 3.4488720148801804
best_a

Current Best Acc,▁▃▅▆▆▇▇▇▇▇██████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▃▂▄▅▅▃▄▆▂▃▃▁▂▂▆▁▄▄▂▂▅▅▅▄█▃▄▂▆▃▄▃▅▄▃▃▃▂▅▆
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▃▅▆▆▇▇▇▇▇██████████████████████████████
test_loss,█▇▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▇▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.84314
Current Best Acc,0.84314
Total Time (hours),0.61491
epoch,75


start
Project Name: Flower102, Run Name BiReact_UniAdapt_thinblock3_prune amount 70% 


Run Start : 2023-04-25 02-19-21
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12256526
Initial accuracy:
Test Accuracy : 1.2%, Test Loss: 4.63739949464798
best_acc.pth saved!
Test Accuracy : 1.1%, Test Loss: 4.631570756435394
epoch: 1 average loss: 4.311
Test Accuracy : 23.0%, Test Loss: 3.9460577368736267
best_acc.pth saved!
Epoch Time (Training + Test) = 14.92 seconds
epoch: 2 average loss: 3.640
Test Accuracy : 37.1%, Test Loss: 3.4122750759124756
bes

Current Best Acc,▁▃▅▆▆▇▇▇▇▇▇█████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▁▆▇▆▄▃▄▇▃▆▇▄▅▃▆▃▃▅▄▄▃▇▄▅▆▄█▃▇▃▂▄▂▄▅▅▅▅▄▃
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▃▅▆▆▇▇▇▇███████████▆███████████████████
test_loss,█▇▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▄▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▇▆▅▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.83137
Current Best Acc,0.83137
Total Time (hours),0.61494
epoch,75


start
Project Name: Flower102, Run Name BiReact_UniAdapt_thinblock3_prune amount 80% 


Run Start : 2023-04-25 02-38-14
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12256526
Initial accuracy:
Test Accuracy : 1.1%, Test Loss: 4.638510376214981
best_acc.pth saved!
Test Accuracy : 1.2%, Test Loss: 4.637084871530533
best_acc.pth saved!
epoch: 1 average loss: 4.313
Test Accuracy : 23.9%, Test Loss: 3.9500784128904343
best_acc.pth saved!
Epoch Time (Training + Test) = 14.74 seconds
epoch: 2 average loss: 3.619
Test Accuracy : 39.8%, Test Loss: 3

Current Best Acc,▁▃▅▆▆▇▇▇▇▇██████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▁▅█▄▆▄█▆▄▅▇▇▄▅▇▆▄▄▄▅▅▄▄▄▃▇▅▃▄█▄▂▄▄▅▄█▃▂▃
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▃▅▆▆▇▇▇▇███████████▇▇▇▇▇███████████████
test_loss,█▇▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▄▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
training_loss,█▇▆▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▄▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
Best Accuracy,0.82843
Current Best Acc,0.82843
Total Time (hours),0.61356
epoch,75


start
Project Name: Flower102, Run Name BiReact_UniAdapt_thinblock3_prune amount 90% 


Run Start : 2023-04-25 02-57-04
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12256526
Initial accuracy:
Test Accuracy : 1.2%, Test Loss: 4.639262080192566
best_acc.pth saved!
Test Accuracy : 1.4%, Test Loss: 4.637546122074127
best_acc.pth saved!
epoch: 1 average loss: 4.308
Test Accuracy : 21.9%, Test Loss: 3.9550316780805588
best_acc.pth saved!
Epoch Time (Training + Test) = 14.79 seconds
epoch: 2 average loss: 3.642
Test Accuracy : 36.4%, Test Loss: 3

## Only adapt Headout


In [4]:
for prune_amount in [0,0.7,0.8,0.9,0.95]:

    net = birealnet18_uniadapt()
    net.load_state_dict(new_dict,strict= False)

    adaptnet = uniAdapt_Net_React([64,128,256,512],[64,128,256,512],block = thinBlock3)
    x = torch.rand((3,3,224,224))
    net.freeze()
    net.uniAdapt = True
    net.uniAdaptNet = adaptnet
    net.adaptHeadOnly = True
    net.head = nn.Linear(512,102)
    net.head.requires_grad = True
    net.head.train()

    trainer = Trainer(net,model_name=f'BiReact_UniAdapt_thinblock3_prune amount {prune_amount*100:.0f}%_AdaptHeadOnly',project_name='Flower102',binarise=False,classes= list(set(Flower_Train._labels)))
    trainer.lr = 1e-3
    trainer.batch_size = 64
    trainer.epochs =75
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.amount = [0.7,prune_amount]
    trainer.model_to_prune = net.uniAdaptNet
    params = nn.ModuleList([net.head,net.uniAdaptNet]).parameters()
    trainer.set_scheduler(None)
    trainer.set_optimizer(torch.optim.Adam,params,lr = trainer.lr)
    prune_strat = None if prune_amount == 0 else 'oneshot'
    trainer.train(Flower_TrainDL,Flower_ValidDL,prune_strat=prune_strat)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]}]}


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: johnny_suu. Use `wandb login --relogin` to force relogin


start
Project Name: Flower102, Run Name BiReact_UniAdapt_thinblock3_prune amount 0%_AdaptHeadOnly 


Run Start : 2023-04-25 21-03-31
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12204302
Initial accuracy:
Test Accuracy : 1.1%, Test Loss: 4.63702055811882
best_acc.pth saved!
Test Accuracy : 1.1%, Test Loss: 4.636060833930969
epoch: 1 average loss: 4.055
Test Accuracy : 27.5%, Test Loss: 3.3797394782304764
best_acc.pth saved!
Epoch Time (Training + Test) = 15.95 seconds
epoch: 2 average loss: 2.825
Test Accuracy : 42.2%, Test Loss: 2.5636143

Current Best Acc,▁▃▆▇████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),█▄▃▃▄▃▃▃▄▄▂▁▂▂▂▁▂▁▁▂▂▁▂▁▂▁▂▁▁▂▂▂▁▂▂▁▂▂▁▁
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▃▆▇█▇▇█▇▇██████████████████████████████
test_loss,█▆▃▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▇▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.81176
Current Best Acc,0.81176
Total Time (hours),0.63226
epoch,75


start
Project Name: Flower102, Run Name BiReact_UniAdapt_thinblock3_prune amount 70%_AdaptHeadOnly 


Run Start : 2023-04-25 21-22-51
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12204302
Initial accuracy:
Test Accuracy : 1.3%, Test Loss: 4.629646211862564
best_acc.pth saved!
Test Accuracy : 1.8%, Test Loss: 4.630046218633652
best_acc.pth saved!
epoch: 1 average loss: 4.012
Test Accuracy : 33.3%, Test Loss: 3.342120036482811
best_acc.pth saved!
Epoch Time (Training + Test) = 14.80 seconds
epoch: 2 average loss: 2.750
Test Accuracy : 48.0%,

...

start
Project Name: Flower102, Run Name BiReact_UniAdapt_thinblock3_prune amount 80%_AdaptHeadOnly 


Run Start : 2023-04-25 21-41-48
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12204302
Initial accuracy:
Test Accuracy : 1.0%, Test Loss: 4.636337071657181
best_acc.pth saved!
Test Accuracy : 1.0%, Test Loss: 4.643826544284821
epoch: 1 average loss: 4.068
Test Accuracy : 32.8%, Test Loss: 3.387940376996994
best_acc.pth saved!
Epoch Time (Training + Test) = 15.14 seconds
epoch: 2 average loss: 2.826
Test Accuracy : 44.4%, Test Loss: 2.505334

Current Best Acc,▁▄▆▇▇▇▇█████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▃▃▃▃▂▄▂▂▃▁▃▃▂▁▂▁▂▂▂█▅▅▅▇▇▆▄▅▄▆█▅▃▂▂▃▃▃▃▃
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▄▆▇▇▇▇███████████▇█▇███████████████████
test_loss,█▆▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▇▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.8049
Current Best Acc,0.8049
Total Time (hours),0.6342
epoch,75


start
Project Name: Flower102, Run Name BiReact_UniAdapt_thinblock3_prune amount 90%_AdaptHeadOnly 


Run Start : 2023-04-25 22-01-20
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12204302
Initial accuracy:
Test Accuracy : 0.6%, Test Loss: 4.636153310537338
best_acc.pth saved!
Test Accuracy : 0.6%, Test Loss: 4.645907819271088
epoch: 1 average loss: 4.072
Test Accuracy : 31.4%, Test Loss: 3.3764660954475403
best_acc.pth saved!
Epoch Time (Training + Test) = 14.97 seconds
epoch: 2 average loss: 2.816
Test Accuracy : 48.0%, Test Loss: 2.47752

Current Best Acc,▁▄▆▇▇███████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▂▄▃▄▄▄▃▄▃▄▄▄▃▄▄▄▄▃▄▄▃▃▂▁▁▂▁▂▃▁▆▇▅▇▆█▆█▇▇
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▄▆▇▇███████████████▇███████████████████
test_loss,█▆▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▇▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.81569
Current Best Acc,0.81569
Total Time (hours),0.62751
epoch,75


start
Project Name: Flower102, Run Name BiReact_UniAdapt_thinblock3_prune amount 95%_AdaptHeadOnly 


Run Start : 2023-04-25 22-20-38
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12204302
Initial accuracy:
Test Accuracy : 1.1%, Test Loss: 4.633679777383804
best_acc.pth saved!
Test Accuracy : 1.1%, Test Loss: 4.639219909906387
epoch: 1 average loss: 4.068
Test Accuracy : 28.5%, Test Loss: 3.4129266291856766
best_acc.pth saved!
Epoch Time (Training + Test) = 14.71 seconds
epoch: 2 average loss: 2.806
Test Accuracy : 48.9%, Test Loss: 2.42294

In [8]:
for prune_amount in [0.99]:

    net = birealnet18_uniadapt()
    net.load_state_dict(new_dict,strict= False)

    adaptnet = uniAdapt_Net_React([64,128,256,512],[64,128,256,512],block = thinBlock3)
    x = torch.rand((3,3,224,224))
    net.freeze()
    net.uniAdapt = True
    net.uniAdaptNet = adaptnet
    net.head.requires_grad = True
    net.head.train()

    trainer = Trainer(net,model_name=f'BiReact_UniAdapt_thinblock3_prune amount {prune_amount*100:.0f}%',project_name='Flower102',binarise=False,classes= list(set(Flower_Train._labels)))
    trainer.lr = 1e-3
    trainer.batch_size = 64
    trainer.epochs =75
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.amount = [prune_amount,prune_amount]
    trainer.model_to_prune = net.uniAdaptNet
    params = nn.ModuleList([net.head,net.uniAdaptNet]).parameters()
    trainer.set_scheduler(None)
    trainer.set_optimizer(torch.optim.Adam,params,lr = trainer.lr)
    prune_strat = None if prune_amount == 0 else 'oneshot'
    trainer.train(Flower_TrainDL,Flower_ValidDL,prune_strat=prune_strat)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]}]}


Current Best Acc,▁▄▆▇▇███████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch time (s),▂▂▂▂▂▃▁█▂▃▂▂▁▁▂▂▂▂▂▁▂▂▂▃▁▃▁▂▁▂▂▁▂▁▂▁▁▁▁▂
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▄▆▇▇███▇▇▇████▇█████████████████▇██████
test_loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▁▂▂▁▂▂▂▂▂▂▂▂
training_loss,█▄▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.7547
Current Best Acc,0.7547
Total Time (hours),4.09954
epoch,150


start
Project Name: Flower102, Run Name BiReact_UniAdapt_thinblock3_prune amount 99% 


Run Start : 2023-04-26 02-51-09
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12256526
Initial accuracy:
Test Accuracy : 1.1%, Test Loss: 4.641756385564804
best_acc.pth saved!
Test Accuracy : 0.7%, Test Loss: 4.6347969472408295
epoch: 1 average loss: 4.322
Test Accuracy : 21.6%, Test Loss: 3.9664257764816284
best_acc.pth saved!
Epoch Time (Training + Test) = 14.82 seconds
epoch: 2 average loss: 3.630
Test Accuracy : 33.1%, Test Loss: 3.4205366522073746
b

# Stanford Cars

In [10]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop((224,224)),
    # transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean,std)
])
valid_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize(mean,std)
])

Cars_Train = torchvision.datasets.StanfordCars(root = './data/StanfordCars/',split='train',download=True,transform= train_transform)
Cars_Valid = torchvision.datasets.StanfordCars(root = './data/StanfordCars/',split='test',download=True,transform= valid_transform)

Cars_TrainDL = DataLoader(Cars_Train,batch_size=64,shuffle=True,num_workers=8)
Cars_ValidDL = DataLoader(Cars_Valid,batch_size=64,num_workers=8)

In [12]:
len(Cars_Train.classes)

196

## Regular

In [ ]:
net = birealnet18_uniadapt()
net.load_state_dict(new_dict,strict= False)

_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])

In [13]:
net.fc = nn.Linear(512,196)
trainer = Trainer(net,model_name='BiReact_Finetune_1e-3',project_name='StandfordCars',binarise=False,classes= Cars_Train.classes)
trainer.lr = 1e-3
trainer.batch_size = 64
trainer.epochs =150
trainer.epoch_chkpts = []
trainer.start_epoch = 0
# params = nn.ModuleList([BNN.head_bn,BNN.head]).parameters()
trainer.set_scheduler(None)
trainer.set_optimizer(torch.optim.Adam,lr = trainer.lr)
trainer.train(Cars_TrainDL,Cars_ValidDL)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]}]}


Current Best Acc,▁▁▁▂▂▂▃▃▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),█▄▅▂▅▄▅▃▃▂▂▂▁▂▂▃▂▂▂▃▂▂▂▁▃▂▂▁▂▃▂▁▂▂▂▁▂▁▁▁
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▁▁▂▂▂▃▃▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█▇██████████
test_loss,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
training_loss,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁
Best Accuracy,0.68039
Current Best Acc,0.68039
Total Time (hours),0.61949
epoch,75


start
Project Name: StandfordCars, Run Name BiReact_Finetune_1e-3 


Run Start : 2023-04-25 11-06-23
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 150
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 11396522
Initial accuracy:
Test Accuracy : 0.4%, Test Loss: 5.287807181477547
best_acc.pth saved!
Test Accuracy : 0.6%, Test Loss: 5.29047797975086
best_acc.pth saved!
epoch: 1 average loss: 5.181
Test Accuracy : 4.8%, Test Loss: 5.061317141093905
best_acc.pth saved!
Epoch Time (Training + Test) = 119.15 seconds
epoch: 2 average loss: 4.958
Test Accuracy : 7.5%, Test Loss: 4.889027489556207
best

In [14]:
net = birealnet18_uniadapt()
net.load_state_dict(new_dict,strict= False)
net.fc = nn.Linear(512,196)

trainer = Trainer(net,model_name='BiReact_Finetune_1e-4',project_name='StandfordCars',binarise=False,classes= list(set(Flower_Train._labels)))
trainer.lr = 1e-4
trainer.batch_size = 64
trainer.epochs =150
trainer.epoch_chkpts = []
trainer.start_epoch = 0
# params = nn.ModuleList([BNN.head_bn,BNN.head]).parameters()
trainer.set_scheduler(None)
trainer.set_optimizer(torch.optim.Adam,lr = trainer.lr)
trainer.train(Cars_TrainDL,Cars_ValidDL)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.0001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]}]}


Current Best Acc,▁▂▄▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇██████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch time (s),▆▆█▇▆▅▅█▃▂▃▄▂▃▃▃▄▅▂▂▂▃▃▃▁▄▃▃▃▂▂▃▃▃▂▂▂▁▂▃
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▂▄▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇█████████████████████
test_loss,█▇▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▇▆▅▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.33018
Current Best Acc,0.33018
Total Time (hours),9.63625
epoch,150


start
Project Name: StandfordCars, Run Name BiReact_Finetune_1e-4 


Run Start : 2023-04-25 15-57-48
start_epoch : 0
initial_lr : 0.0001
batch_size : 64
epochs : 150
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.0001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 11396522
Initial accuracy:
Test Accuracy : 0.3%, Test Loss: 5.286476366221905
best_acc.pth saved!
Test Accuracy : 0.5%, Test Loss: 5.285822724539136
best_acc.pth saved!
epoch: 1 average loss: 5.273
Test Accuracy : 0.9%, Test Loss: 5.254042428637308
best_acc.pth saved!
Epoch Time (Training + Test) = 115.74 seconds
epoch: 2 average loss: 5.240
Test Accuracy : 1.4%, Test Loss: 5.222913692867945
b

In [15]:
net = birealnet18_uniadapt()
net.load_state_dict(new_dict,strict= False)
net.freeze()
net.fc = nn.Linear(512,196)

trainer = Trainer(net,model_name='BiReact_Finetune_HeadOnly',project_name='StandfordCars',binarise=False,classes= list(set(Flower_Train._labels)))
trainer.lr = 1e-4
trainer.batch_size = 64
trainer.epochs =150
trainer.epoch_chkpts = []
trainer.start_epoch = 0
# params = nn.ModuleList([BNN.head_bn,BNN.head]).parameters()
params = net.fc.parameters()
trainer.set_scheduler(None)
trainer.set_optimizer(torch.optim.Adam,params,lr = trainer.lr)
trainer.train(Cars_TrainDL,Cars_ValidDL)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.0001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1]}]}


Current Best Acc,▁▁▂▂▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch time (s),▄▄▇▅▂▄▃▅▃▅▆▇██▇▆▄▄▄▁▇▄▂▂▄▂▃▆▃▃▃▂▄▅▃▄▅▄▂▅
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▁▂▂▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████
test_loss,██▇▆▆▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
training_loss,██▇▇▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
Best Accuracy,0.36811
Current Best Acc,0.36811
Total Time (hours),9.66836
epoch,150


start
Project Name: StandfordCars, Run Name BiReact_Finetune_HeadOnly 


Run Start : 2023-04-25 20-49-58
start_epoch : 0
initial_lr : 0.0001
batch_size : 64
epochs : 150
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.0001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 11396522
Initial accuracy:
Test Accuracy : 0.4%, Test Loss: 5.28710113838315
best_acc.pth saved!
Test Accuracy : 0.5%, Test Loss: 5.288306084890214
best_acc.pth saved!
epoch: 1 average loss: 5.272
Test Accuracy : 1.0%, Test Loss: 5.257618726245941
best_acc.pth saved!
Epoch Time (Training + Test) = 117.84 seconds
epoch: 2 average loss: 5.244
Test Accuracy : 1.4%, Test Loss: 5.23256218622601

KeyboardInterrupt: 

## UniAdapt

In [13]:

for prune_amount in [0,0.9,0.99]:

    net = birealnet18_uniadapt()
    net.load_state_dict(new_dict,strict= False)

    adaptnet = uniAdapt_Net_React([64,128,256,512],[64,128,256,512],block = thinBlock3)
    x = torch.rand((3,3,224,224))
    net.freeze()
    net.head = nn.Linear(512,196)
    net.adaptHeadOnly = True
    net.uniAdapt = True
    net.uniAdaptNet = adaptnet
    net.head.requires_grad_(True)
    net.head.train()

    trainer = Trainer(net,model_name=f'BiReact_UniAdapt_thinblock3_prune amount {prune_amount*100:.0f}%',project_name='StanfordCars',binarise=False,classes= Cars_Train.classes)
    trainer.lr = 1e-4
    trainer.batch_size = 64
    trainer.epochs =75
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.amount = [prune_amount,prune_amount]
    trainer.model_to_prune = net.uniAdaptNet
    params = nn.ModuleList([net.head,net.uniAdaptNet]).parameters()
    trainer.set_scheduler(None)
    trainer.set_optimizer(torch.optim.Adam,params,lr = trainer.lr)
    prune_strat = None if prune_amount == 0 else 'oneshot'
    trainer.train(Cars_TrainDL,Cars_ValidDL,prune_strat=prune_strat)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.0001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]}]}


Current Best Acc,▁▂▃▅▅▆▆▆▇▇▇▇▇▇▇▇▇███████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▇▂▁█▇▇▅▆▅▄▄▅▆▅▄▄▆▄▄▅▄▅▆▅▅▅▆▆▃▄▄▅▄▆▄▆▄▆▄▇
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▂▃▅▅▅▆▆▇▇▇▇▇▇▇▇▇███▂▃▃▃▃▄▄▄▄▄▄▄▄▄▄▄▄▅▅▅
test_loss,█▇▅▃▃▃▂▂▂▂▂▂▂▂▁▁▂▁▁▁▇▆▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
training_loss,█▇▅▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁█▆▆▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄
Best Accuracy,0.41114
Current Best Acc,0.41114
Total Time (hours),5.2147
epoch,75


start
Project Name: StanfordCars, Run Name BiReact_UniAdapt_thinblock3_prune amount 0% 


Run Start : 2023-04-27 09-36-50
start_epoch : 0
initial_lr : 0.0001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.0001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12252524
Initial accuracy:
Test Accuracy : 0.4%, Test Loss: 5.294783137738705
best_acc.pth saved!
Test Accuracy : 0.4%, Test Loss: 5.299826103543478
epoch: 1 average loss: 5.198
Test Accuracy : 2.4%, Test Loss: 5.10400756957039
best_acc.pth saved!
Epoch Time (Training + Test) = 127.63 seconds
epoch: 2 average loss: 4.990
Test Accuracy : 3.9%, Test Loss: 4.951040453381008
be

Current Best Acc,▁▁▂▂▃▃▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▄▃█▇▇▇▇▆▄▂▃▂▁▂▁▂▂▁▂▂▂▂▁▁▁▁▁▁▁▂▂▂▂▁▁▁▂▁▂▁
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▁▂▂▃▃▄▄▄▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇█▇█████████
test_loss,█▇▇▆▆▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,██▇▇▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
Best Accuracy,0.3604
Current Best Acc,0.3604
Total Time (hours),5.21807
epoch,75


start
Project Name: StanfordCars, Run Name BiReact_UniAdapt_thinblock3_prune amount 90% 


Run Start : 2023-04-27 12-15-44
start_epoch : 0
initial_lr : 0.0001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.0001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12252524
Initial accuracy:
Test Accuracy : 0.5%, Test Loss: 5.297224026173353
best_acc.pth saved!
Test Accuracy : 0.4%, Test Loss: 5.3011421362559
epoch: 1 average loss: 5.203
Test Accuracy : 2.9%, Test Loss: 5.111493822128054
best_acc.pth saved!
Epoch Time (Training + Test) = 125.54 seconds
epoch: 2 average loss: 4.995
Test Accuracy : 4.5%, Test Loss: 4.9447480080619695
b

Current Best Acc,▁▂▂▃▃▄▄▅▅▆▆▆▆▇▇▇▇▇██████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▁▃▃█▅▃▇▄▂▅▃▁▅▃▃▂▆▄▄▂▂▃▃▃▃▂▃▄▄▅▂▁▄▄▂▃▃▄▂▂
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▂▂▃▃▄▄▅▅▆▆▆▆▇▇▇████▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇
test_loss,█▇▇▆▅▅▄▄▃▃▃▂▂▂▂▁▁▁▁▁▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
training_loss,██▇▆▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁
Best Accuracy,0.30481
Current Best Acc,0.30481
Total Time (hours),5.24143
epoch,75


start
Project Name: StanfordCars, Run Name BiReact_UniAdapt_thinblock3_prune amount 99% 


Run Start : 2023-04-27 14-55-15
start_epoch : 0
initial_lr : 0.0001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.0001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12252524
Initial accuracy:
Test Accuracy : 0.6%, Test Loss: 5.298496808856726
best_acc.pth saved!
Test Accuracy : 0.5%, Test Loss: 5.302362074927678
epoch: 1 average loss: 5.195
Test Accuracy : 2.5%, Test Loss: 5.099065390844194
best_acc.pth saved!
Epoch Time (Training + Test) = 124.81 seconds
epoch: 2 average loss: 4.987
Test Accuracy : 4.0%, Test Loss: 4.933164524653601


In [ ]:
for prune_amount in [0,0.7,0.8,0.9,0.95]:

    net = birealnet18_uniadapt()
    net.load_state_dict(new_dict,strict= False)

    adaptnet = uniAdapt_Net_React([64,128,256,512],[64,128,256,512],block = thinBlock3)
    x = torch.rand((3,3,224,224))
    net.freeze()
    net.head = nn.Linear(512+512,196)
    net.uniAdapt = True
    net.uniAdaptNet = adaptnet
    net.head.requires_grad = True
    net.head.train()

    trainer = Trainer(net,model_name=f'BiReact_UniAdapt_thinblock3_prune amount {prune_amount*100:.0f}%',project_name='StandfordCars',binarise=False,classes= list(set(Flower_Train._labels)))
    trainer.lr = 1e-3
    trainer.batch_size = 64
    trainer.epochs =75
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.amount = [0.7,prune_amount]
    trainer.model_to_prune = net.uniAdaptNet
    params = nn.ModuleList([net.head,net.uniAdaptNet]).parameters()
    trainer.set_scheduler(None)
    trainer.set_optimizer(torch.optim.Adam,params,lr = trainer.lr)
    prune_strat = None if prune_amount == 0 else 'oneshot'
    trainer.train(Flower_TrainDL,Flower_ValidDL,prune_strat=prune_strat)

# Oxford Pets

In [4]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop((224,224)),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean,std)
])
valid_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize(mean,std)
])

Pets_Train = torchvision.datasets.OxfordIIITPet(root = './data/OxfordIIITPet/',split='trainval',download=True,transform= train_transform)
Pets_Valid = torchvision.datasets.OxfordIIITPet(root = './data/OxfordIIITPet/',split='test',download=True,transform= valid_transform)

Pets_TrainDL = DataLoader(Pets_Train,batch_size=64,shuffle=True,num_workers=8)
Pets_ValidDL = DataLoader(Pets_Valid,batch_size=64,num_workers=8)

## Regular

In [6]:
net = birealnet18_uniadapt()
net.load_state_dict(new_dict,strict= False)
net.freeze()

net.fc = nn.Linear(512,37)

trainer = Trainer(net,model_name='BiReact_Finetune_HeadOnly_1e-4',project_name='OxfordPets',binarise=False,classes=Pets_Train.classes)
trainer.lr = 1e-4
trainer.batch_size = 64
trainer.epochs = 100
trainer.epoch_chkpts = []
trainer.start_epoch = 0
# params = nn.ModuleList([BNN.head_bn,BNN.head]).parameters()
trainer.set_scheduler(None)
trainer.set_optimizer(torch.optim.Adam,params = net.fc.parameters(),lr = trainer.lr)
trainer.train(Pets_TrainDL,Pets_ValidDL)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.0001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1]}]}


Current Best Acc,▁▆▇▇▇███████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▄▂▆▅▅▄▄▇▄▃▆▆▁▅▃▇▅▃▅▆▅▄▅█▄▄▄▅▅▆▄▆█▃▄▆▅▅▃█
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▆▇▇▇██▇███▇████████▇▇██████████████████
test_loss,█▃▂▂▂▁▁▂▁▁▁▂▁▂▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▅▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▄▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.76261
Current Best Acc,0.76261
Total Time (hours),1.80521
epoch,75


start
Project Name: OxfordPets, Run Name BiReact_Finetune_HeadOnly_1e-4 


Run Start : 2023-04-26 15-41-08
start_epoch : 0
initial_lr : 0.0001
batch_size : 64
epochs : 100
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.0001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 11314955
Initial accuracy:
Test Accuracy : 2.5%, Test Loss: 3.6177223386435675
best_acc.pth saved!
Test Accuracy : 2.8%, Test Loss: 3.612777623636969
best_acc.pth saved!
epoch: 1 average loss: 3.587
Test Accuracy : 7.7%, Test Loss: 3.5405060587258173
best_acc.pth saved!
Epoch Time (Training + Test) = 42.81 seconds
epoch: 2 average loss: 3.529
Test Accuracy : 14.7%, Test Loss: 3.470382036

In [7]:
net = birealnet18_uniadapt()
net.load_state_dict(new_dict,strict= False)
net.freeze()

net.fc = nn.Linear(512,37)

trainer = Trainer(net,model_name='BiReact_Finetune_HeadOnly_1e-3',project_name='OxfordPets',binarise=False,classes=Pets_Train.classes)
trainer.lr = 1e-3
trainer.batch_size = 64
trainer.epochs = 100
trainer.epoch_chkpts = []
trainer.start_epoch = 0
# params = nn.ModuleList([BNN.head_bn,BNN.head]).parameters()
trainer.set_scheduler(None)
trainer.set_optimizer(torch.optim.Adam,params = net.fc.parameters(),lr = trainer.lr)
trainer.train(Pets_TrainDL,Pets_ValidDL)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1]}]}


Current Best Acc,▁▂▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇██████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▂▂▂▂▄▃▃▃▇▂▁▂▅▃▃▃▃▃▃▂▂▂▃▂▃▃▄▃▂█▂▄▁▃▂▂▁▃▂▂
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▂▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇███████████████████████
test_loss,██▇▇▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
training_loss,██▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
Best Accuracy,0.70319
Current Best Acc,0.70319
Total Time (hours),2.36762
epoch,100


start
Project Name: OxfordPets, Run Name BiReact_Finetune_HeadOnly_1e-3 


Run Start : 2023-04-26 16-53-14
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 100
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 11314955
Initial accuracy:
Test Accuracy : 2.6%, Test Loss: 3.6155642682108384
best_acc.pth saved!
Test Accuracy : 2.4%, Test Loss: 3.6150104670689025
epoch: 1 average loss: 3.357
Test Accuracy : 49.0%, Test Loss: 2.980874045141812
best_acc.pth saved!
Epoch Time (Training + Test) = 42.48 seconds
epoch: 2 average loss: 2.898
Test Accuracy : 54.8%, Test Loss: 2.53715345571781
best_acc.pth sa

KeyboardInterrupt: 

In [6]:
net = birealnet18_uniadapt()
net.load_state_dict(new_dict,strict= False)
net.fc = nn.Linear(512,37)

trainer = Trainer(net,model_name='BiReact_Finetune_1e-4',project_name='OxfordPets',binarise=False,classes=Pets_Train.classes)
trainer.lr = 1e-4
trainer.batch_size = 64
trainer.epochs =150
trainer.epoch_chkpts = []
trainer.start_epoch = 0
# params = nn.ModuleList([BNN.head_bn,BNN.head]).parameters()
trainer.set_scheduler(None)
trainer.set_optimizer(torch.optim.Adam,lr = trainer.lr)
trainer.train(Pets_TrainDL,Pets_ValidDL)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.0001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]}]}


Current Best Acc,▁▃▆▇▇▇██████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▄▇▇▆▄▅▇▄▅█▅▄▄▅▄▄▃▅▃▅▅▃▅▃▅▅▆▂▃▅▆▅▃▅▁▃▅▅▅▂
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▃▆▇▇▇██████████████▆███████████████████
test_loss,█▆▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▇▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.80196
Current Best Acc,0.80196
Total Time (hours),0.60284
epoch,75


start
Project Name: OxfordPets, Run Name BiReact_Finetune_1e-4 


Run Start : 2023-04-25 22-41-42
start_epoch : 0
initial_lr : 0.0001
batch_size : 64
epochs : 150
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.0001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 11314955
Initial accuracy:
Test Accuracy : 2.9%, Test Loss: 3.6196420151611854
best_acc.pth saved!
Test Accuracy : 2.9%, Test Loss: 3.6179456094215654
epoch: 1 average loss: 3.586
Test Accuracy : 8.7%, Test Loss: 3.5349916260817955
best_acc.pth saved!
Epoch Time (Training + Test) = 48.26 seconds
epoch: 2 average loss: 3.516
Test Accuracy : 17.8%, Test Loss: 3.4468089794290475
best_acc.pth saved!


In [7]:
net = birealnet18_uniadapt()
net.load_state_dict(new_dict,strict= False)
net.fc = nn.Linear(512,37)

trainer = Trainer(net,model_name='BiReact_Finetune_1e-3',project_name='OxfordPets',binarise=False,classes=Pets_Train.classes)
trainer.lr = 1e-3
trainer.batch_size = 64
trainer.epochs =150
trainer.epoch_chkpts = []
trainer.start_epoch = 0
# params = nn.ModuleList([BNN.head_bn,BNN.head]).parameters()
trainer.set_scheduler(None)
trainer.set_optimizer(torch.optim.Adam,lr = trainer.lr)
trainer.train(Pets_TrainDL,Pets_ValidDL)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]}]}


Current Best Acc,▁▃▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch time (s),▁▂▅▅██▆▇▅▅▆█▄▅▄▅▆▆▅▅▃▃▄▃▃▄▃▃▅▃▄▄▃▃▃▃▃▃▃▃
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▃▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇██████████████████████
test_loss,██▆▅▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,██▇▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.74898
Current Best Acc,0.74898
Total Time (hours),4.14873
epoch,150


start
Project Name: OxfordPets, Run Name BiReact_Finetune_1e-3 


Run Start : 2023-04-26 00-47-12
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 150
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 11314955
Initial accuracy:
Test Accuracy : 1.9%, Test Loss: 3.632601659873436
best_acc.pth saved!
Test Accuracy : 2.1%, Test Loss: 3.6353217034504333
best_acc.pth saved!
epoch: 1 average loss: 3.232
Test Accuracy : 46.2%, Test Loss: 2.478688799101731
best_acc.pth saved!
Epoch Time (Training + Test) = 49.22 seconds
epoch: 2 average loss: 2.135
Test Accuracy : 54.7%, Test Loss: 1.7342118641425823
bes

## Finetune 

In [5]:
for prune_amount in [0,0.7,0.8,0.9,0.95,0.99]:

    net = birealnet18_uniadapt()
    net.load_state_dict(new_dict,strict= False)

    adaptnet = uniAdapt_Net_React([64,128,256,512],[64,128,256,512],block = thinBlock3)
    x = torch.rand((3,3,224,224))
    net.freeze()
    net.head = nn.Linear(1024,37)
    net.uniAdapt = True
    net.uniAdaptNet = adaptnet
    net.head.requires_grad = True
    net.head.train()

    trainer = Trainer(net,model_name=f'BiReact_UniAdapt_thinblock3_prune amount {prune_amount*100:.0f}%',project_name='OxfordPets',binarise=False,classes= Pets_Train.classes)
    trainer.lr = 1e-3
    trainer.batch_size = 64
    trainer.epochs =75
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.amount = [0.7,prune_amount]
    trainer.model_to_prune = net.uniAdaptNet
    params = nn.ModuleList([net.head,net.uniAdaptNet]).parameters()
    trainer.set_scheduler(None)
    trainer.set_optimizer(torch.optim.Adam,params,lr = trainer.lr)
    prune_strat = None if prune_amount == 0 else 'oneshot'
    trainer.train(Pets_TrainDL,Pets_ValidDL,prune_strat = prune_strat)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]}]}


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: johnny_suu. Use `wandb login --relogin` to force relogin


start
Project Name: OxfordPets, Run Name BiReact_UniAdapt_thinblock3_prune amount 0% 


Run Start : 2023-04-26 10-09-44
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12189901
Initial accuracy:
Test Accuracy : 1.9%, Test Loss: 3.623230140784691
best_acc.pth saved!
Test Accuracy : 1.0%, Test Loss: 3.631646226192343
epoch: 1 average loss: 2.411
Test Accuracy : 51.4%, Test Loss: 1.6574093247282093
best_acc.pth saved!
Epoch Time (Training + Test) = 45.55 seconds
epoch: 2 average loss: 1.487
Test Accuracy : 55.8%, Test Loss: 1.4023524491951382
be

Current Best Acc,▁▆▇▇████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▆▅▄▄▇█▆▇▅▅▃▃▃▃▃▃▃▄▃▃▁▃▁▁▂▁▂▁▂▂▂▁▂▁▁▂▂▁▂▂
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▆▇███▇█████████████████████████████████
test_loss,█▃▂▁▁▁▂▁▁▂▁▂▁▂▁▁▁▂▁▁▁▁▁▁▂▂▁▂▂▂▂▁▁▁▂▂▂▂▂▂
training_loss,█▅▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.76424
Current Best Acc,0.76424
Total Time (hours),1.84577
epoch,75


start
Project Name: OxfordPets, Run Name BiReact_UniAdapt_thinblock3_prune amount 70% 


Run Start : 2023-04-26 11-05-58
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12189901
Initial accuracy:
Test Accuracy : 2.7%, Test Loss: 3.62320312549328
best_acc.pth saved!
Test Accuracy : 2.6%, Test Loss: 3.6364061380254813
epoch: 1 average loss: 2.452
Test Accuracy : 52.7%, Test Loss: 1.631826805657354
best_acc.pth saved!
Epoch Time (Training + Test) = 43.04 seconds
epoch: 2 average loss: 1.528
Test Accuracy : 58.2%, Test Loss: 1.38102205046292
best

...

start
Project Name: OxfordPets, Run Name BiReact_UniAdapt_thinblock3_prune amount 90% 


Run Start : 2023-04-26 12-56-00
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12189901
Initial accuracy:
Test Accuracy : 2.5%, Test Loss: 3.6312076798800765
best_acc.pth saved!
Test Accuracy : 2.9%, Test Loss: 3.6286591085894355
best_acc.pth saved!
epoch: 1 average loss: 2.411
Test Accuracy : 46.2%, Test Loss: 1.9347772706171562
best_acc.pth saved!
Epoch Time (Training + Test) = 43.60 seconds
epoch: 2 average loss: 1.470
Test Accuracy : 59.8%, Test Loss

Current Best Acc,▁▅▇▇▇▇██████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▃▅▃▃▄▅▆▃▂▃▁▆▃▃▄▂▃▂▃▄▄▁▃▃▆█▄▇▅▃▃▂▅▃▅▄▃▄▃▄
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▅▇▇▇▇████████████▇█▇███████████████████
test_loss,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▅▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.75879
Current Best Acc,0.75879
Total Time (hours),1.80378
epoch,75


start
Project Name: OxfordPets, Run Name BiReact_UniAdapt_thinblock3_prune amount 95% 


Run Start : 2023-04-26 13-51-04
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12189901
Initial accuracy:
Test Accuracy : 3.1%, Test Loss: 3.617776796735566
best_acc.pth saved!
Test Accuracy : 2.3%, Test Loss: 3.613582656301301
epoch: 1 average loss: 2.394
Test Accuracy : 48.5%, Test Loss: 1.788534601186884
best_acc.pth saved!
Epoch Time (Training + Test) = 43.05 seconds
epoch: 2 average loss: 1.463
Test Accuracy : 62.5%, Test Loss: 1.260028735317033
bes

Current Best Acc,▁▅▇▇▇███████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▄▃▄▆▅▃█▃▄▄▁▂▃▂▃▄▄▃▃▄▄▃▄▄▃▃▃▃▃▅▁▃▆▅▃▃▆▂▃▃
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▅▇▇▇███████████████▇███████████████████
test_loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▅▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.76451
Current Best Acc,0.76451
Total Time (hours),1.80235
epoch,75


start
Project Name: OxfordPets, Run Name BiReact_UniAdapt_thinblock3_prune amount 99% 


Run Start : 2023-04-26 14-46-03
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12189901
Initial accuracy:
Test Accuracy : 3.5%, Test Loss: 3.625403280915885
best_acc.pth saved!
Test Accuracy : 2.5%, Test Loss: 3.633056932482226
epoch: 1 average loss: 2.420
Test Accuracy : 50.2%, Test Loss: 1.7298638543178295
best_acc.pth saved!
Epoch Time (Training + Test) = 42.91 seconds
epoch: 2 average loss: 1.511
Test Accuracy : 58.8%, Test Loss: 1.3227477818727493
b

## UniAdapt Head only    

In [9]:
for prune_amount in [0,0.7,0.8,0.9,0.95,0.99]:

    net = birealnet18_uniadapt()
    net.load_state_dict(new_dict,strict= False)

    adaptnet = uniAdapt_Net_React([64,128,256,512],[64,128,256,512],block = thinBlock3)
    x = torch.rand((3,3,224,224))
    net.freeze()
    net.uniAdapt = True
    net.uniAdaptNet = adaptnet
    net.adaptHeadOnly = True
    net.head = nn.Linear(512,37)
    net.head.requires_grad = True
    net.head.train()

    trainer = Trainer(net,model_name=f'BiReact_UniAdapt_thinblock3_prune amount {prune_amount*100:.0f}%_AdaptHeadOnly',project_name='OxfordPets',binarise=False,classes= Pets_Train.classes)
    trainer.lr = 1e-3
    trainer.batch_size = 64
    trainer.epochs =75
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.amount = [0.7,prune_amount]
    trainer.model_to_prune = net.uniAdaptNet
    params = nn.ModuleList([net.head,net.uniAdaptNet]).parameters()
    trainer.set_scheduler(None)
    trainer.set_optimizer(torch.optim.Adam,params,lr = trainer.lr)
    prune_strat = None if prune_amount == 0 else 'oneshot'
    trainer.train(Pets_TrainDL,Pets_ValidDL,prune_strat = prune_strat)

No lr scheduler
Optimizer Set to New: {'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]}]}


Current Best Acc,▁▃▅▆▆▇▇▇▇▇██████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▁▄▄▄▃▆▄▅▃▄▃▅▄▅▅▂▁▅▂▃▂█▁▁▂▁▃▃▂▄▃▄▂▃▂▃▃▁▂▅
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▃▅▆▆▇▇▇▇███████████▁▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
test_loss,█▇▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁█▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆
training_loss,█▇▆▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁█▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆
Best Accuracy,0.82157
Current Best Acc,0.82157
Total Time (hours),0.61438
epoch,75


start
Project Name: OxfordPets, Run Name BiReact_UniAdapt_thinblock3_prune amount 0%_AdaptHeadOnly 


Run Start : 2023-04-26 03-10-05
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12170957
Initial accuracy:
Test Accuracy : 1.9%, Test Loss: 3.628069515885978
best_acc.pth saved!
Test Accuracy : 1.9%, Test Loss: 3.632398580682689
epoch: 1 average loss: 2.419
Test Accuracy : 47.4%, Test Loss: 1.7995489969335754
best_acc.pth saved!
Epoch Time (Training + Test) = 44.49 seconds
epoch: 2 average loss: 1.501
Test Accuracy : 57.2%, Test Loss: 1.44367

Current Best Acc,▁▅▇▇▇███████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▅▇▃▅▅▅▆▃▆▃▇▄▃▄▅▇▁▅█▄▆▁▇▅▄▄▅▄▅▇▄▃▃▆▆▄▇▃▅▄
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▅▇▇▆█▇█▇██▇████████████████████████████
test_loss,█▃▂▂▃▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▁▁▂▂▂▂▂▂▂▁▂▁▂
training_loss,█▅▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.74543
Current Best Acc,0.74543
Total Time (hours),1.83838
epoch,75


start
Project Name: OxfordPets, Run Name BiReact_UniAdapt_thinblock3_prune amount 70%_AdaptHeadOnly 


Run Start : 2023-04-26 04-06-13
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12170957
Initial accuracy:
Test Accuracy : 2.6%, Test Loss: 3.6270073282307593
best_acc.pth saved!
Test Accuracy : 2.0%, Test Loss: 3.6351404477810036
epoch: 1 average loss: 2.424
Test Accuracy : 47.0%, Test Loss: 1.7838497922338288
best_acc.pth saved!
Epoch Time (Training + Test) = 44.27 seconds
epoch: 2 average loss: 1.503
Test Accuracy : 52.0%, Test Loss: 1.70

...

start
Project Name: OxfordPets, Run Name BiReact_UniAdapt_thinblock3_prune amount 90%_AdaptHeadOnly 


Run Start : 2023-04-26 05-58-51
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12170957
Initial accuracy:
Test Accuracy : 2.9%, Test Loss: 3.6151426701710143
best_acc.pth saved!
Test Accuracy : 2.8%, Test Loss: 3.627746351834001
epoch: 1 average loss: 2.416
Test Accuracy : 46.0%, Test Loss: 1.861956820919596
best_acc.pth saved!
Epoch Time (Training + Test) = 43.90 seconds
epoch: 2 average loss: 1.484
Test Accuracy : 57.2%, Test Loss: 1.4370

Current Best Acc,▁▅▇▇▇███████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▅▂▇▃▇▃▂▃▂▂▂▃▅▆▂▄▇▁▃▂▄▁▄▄▇▅▂▂▃▂▃▆▆▄▃▂▄▄▆█
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▅▇▇▇▇▇█▇██▇████▇███▇▇██████████████████
test_loss,█▄▂▁▁▂▂▁▂▁▁▂▁▁▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.75252
Current Best Acc,0.75252
Total Time (hours),1.84162
epoch,75


start
Project Name: OxfordPets, Run Name BiReact_UniAdapt_thinblock3_prune amount 95%_AdaptHeadOnly 


Run Start : 2023-04-26 06-55-04
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12170957
Initial accuracy:
Test Accuracy : 2.4%, Test Loss: 3.615316966484333
best_acc.pth saved!
Test Accuracy : 1.1%, Test Loss: 3.6224358780630705
epoch: 1 average loss: 2.411
Test Accuracy : 50.9%, Test Loss: 1.723661006010812
best_acc.pth saved!
Epoch Time (Training + Test) = 44.86 seconds
epoch: 2 average loss: 1.495
Test Accuracy : 55.6%, Test Loss: 1.5206

Current Best Acc,▁▆▇▇████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch time (s),▃▇▄▂▂▄▄▂▇▂▄▄▂█▃▃▅▃▂▄▃▃▇▃▁▅▅▁▄▄▅▃▃▃▃▄▅▅▂▃
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▆▇▇█▇█▇▇███████████▇▇██████████████████
test_loss,█▃▂▂▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_loss,█▅▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best Accuracy,0.74598
Current Best Acc,0.74598
Total Time (hours),1.84113
epoch,75


start
Project Name: OxfordPets, Run Name BiReact_UniAdapt_thinblock3_prune amount 99%_AdaptHeadOnly 


Run Start : 2023-04-26 07-51-16
start_epoch : 0
initial_lr : 0.001
batch_size : 64
epochs : 75
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
seed : None
model_architecture : <class 'NN_Thesis.models.ReactNet.birealnet18_uniadapt'>
binerised_training : False
Number of Elements : 12170957
Initial accuracy:
Test Accuracy : 3.2%, Test Loss: 3.6162545722106407
best_acc.pth saved!
Test Accuracy : 3.5%, Test Loss: 3.616521321494004
best_acc.pth saved!
epoch: 1 average loss: 2.399
Test Accuracy : 48.3%, Test Loss: 1.823261420274603
best_acc.pth saved!
Epoch Time (Training + Test) = 44.21 seconds
epoch: 2 average loss: 1.494
Test Accuracy : 59.3

## Regular Adapter